# Introduction

In this notebook, we carry out the Transfer Learning Experiments.
The other notebooks, as well as the thesis can be found at https://github.com/fridowicke/infant_cries

# Imports

In [4]:
import tensorflow as tf
import numpy as np
# from google.colab import drive
import json

from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

# Loading the Data

## Mount the Drive

In [ ]:
drive.mount('/content/drive')

## Save / Load Processed Data

In [ ]:
#We use json to save the spectrograms and f0s of the produced data in order to avoid computing them every time we use the notebook.
#Helper functions
def save(data, filename):
    if type(data) is np.ndarray:
      data = data.tolist()
    with open(filename, 'w') as f:
        json.dump(data, f, ensure_ascii=False)
        
def load(filename):
    with open(filename) as data:
        x = json.load(data)
    return np.array(x)

In [ ]:
#Load the Data
specs_train = load("/content/drive/My Drive/infant_cries/data_js_final/specs_train.json")
y_train = load("/content/drive/My Drive/infant_cries/data_js_final/y_train.json")
specs_dev_ger = load("/content/drive/My Drive/infant_cries/data_js_final/specs_dev_ger.json")
specs_dev_jap = load("/content/drive/My Drive/infant_cries/data_js_final/specs_dev_jap.json")
specs_test_ger = load("/content/drive/My Drive/infant_cries/data_js_final/specs_dev_ger.json")
specs_test_jap = load("/content/drive/My Drive/infant_cries/data_js_final/specs_dev_jap.json")
specs_eval = load("/content/drive/My Drive/infant_cries/data_js_final/specs_eval.json")
X_dev_ger = load("/content/drive/My Drive/infant_cries/data_js_final/X_dev_ger.json")
X_dev_jap = load("/content/drive/My Drive/infant_cries/data_js_final/X_dev_jap.json")
X_test_ger = load("/content/drive/My Drive/infant_cries/data_js_final/X_dev_ger.json")
X_test_jap = load("/content/drive/My Drive/infant_cries/data_js_final/X_dev_jap.json")
X_eval = load("/content/drive/My Drive/infant_cries/data_js_final/X_eval.json")


## Create Datasets

In [ ]:
#Size of the Dataset
n, height, width = specs_train.shape
n_dev_ger = specs_dev_ger.shape[0]
n_dev_jap = specs_dev_jap.shape[0]
n_test_ger = specs_test_ger.shape[0]
n_test_jap = specs_test_jap.shape[0]
n_eval = specs_eval.shape[0]

#Create the ys for dev and test set
y_dev_ger = np.zeros(n_dev_ger)
y_dev_jap = np.ones(n_dev_jap)
y_test_ger = np.zeros(n_test_ger)
y_test_jap = np.ones(n_test_jap)

#Reshape spectrograms for train, dev, test, eval
specs_train = specs_train.reshape([n,height,width,1])
specs_dev_ger = specs_dev_ger.reshape([n_dev_ger,height,width,1])
specs_dev_jap = specs_dev_jap.reshape([n_dev_jap,height,width,1])
specs_test_ger = specs_test_ger.reshape([n_test_ger,height,width,1])
specs_test_jap = specs_test_jap.reshape([n_test_jap,height,width,1])
specs_eval = specs_eval.reshape([n_eval,height,width,1])

#Create mixed dev set
specs_dev = np.zeros((2*n_dev_jap, height, width, 1))
specs_dev[:n_dev_jap] = specs_dev_jap
specs_dev[n_dev_jap:] = np.vstack((specs_dev_ger,specs_dev_ger,specs_dev_ger,specs_dev_ger))[:n_dev_jap]
y_dev = np.zeros(2*n_dev_jap)
y_dev[:n_dev_jap] = 1

#Create mixed test set
specs_test = np.zeros((2*n_test_jap, height, width, 1))
specs_test[:n_test_jap] = specs_test_jap
specs_test[n_test_jap:] = np.vstack((specs_test_ger,specs_test_ger,specs_test_ger,specs_test_ger))[:n_test_jap]
y_test = np.zeros(2*n_test_jap)
y_test[:n_test_jap] = 1

#Create the eval set
y_eval = np.zeros(n_eval)

#Set the Class Names
class_names = ["German", "Japanese"]

# Pretrained TRILL

## Create Dataset and function to pretrain TRILL

In [ ]:
specs_train_resnet = tf.tile(specs_train, [1,1,1,3])
specs_dev_resnet = tf.tile(specs_dev, [1,1,1,3])
specs_dev_ger_resnet = tf.tile(specs_dev_ger, [1,1,1,3])
specs_dev_jap_resnet = tf.tile(specs_dev_jap, [1,1,1,3])

specs_test_resnet = tf.tile(specs_test, [1,1,1,3])
specs_test_ger_resnet = tf.tile(specs_test_ger, [1,1,1,3])
specs_test_jap_resnet = tf.tile(specs_test_jap, [1,1,1,3])

specs_eval_resnet = tf.tile(specs_eval, [1,1,1,3])
input_shape  = (height, width,3)

In [5]:
import tensorflow_hub as hub

In [17]:
def pretrain_trill():
  input = layers.Input(shape=(16000))
  m = hub.KerasLayer('https://tfhub.dev/google/trillsson5/1')
  # NOTE: Audio should be floats in [-1, 1], sampled at 16kHz. Model input is of
  # the shape [batch size, time].
  # audio_samples = tf.zeros([3, 64000])

  embeddings = m(input)['embedding']

  x = layers.Flatten()(embeddings)
  x = layers.Dense(1000, activation='relu')(x)
  predictions = layers.Dense(2, activation='softmax')(x)

  trill_pretrained = tf.keras.Model(inputs = m.input, outputs = predictions)
  trill_pretrained.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
  return trill_pretrained



In [18]:
model = pretrain_trill()

2023-02-04 11:38:34.561256: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2023-02-04 11:38:34.561601: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2023-02-04 11:38:34.561735: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library


ValueError: Exception encountered when calling layer "keras_layer_4" (type KerasLayer).

in user code:

    File "/home/tim/Documents/dfki/shared_projects/infant_cries/venv/lib/python3.8/site-packages/tensorflow_hub/keras_layer.py", line 237, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (3 total):
        * <tf.Tensor 'inputs:0' shape=(None, None, 200) dtype=float32>
        * False
        * None
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (3 total):
        * TensorSpec(shape=(None, None), dtype=tf.float32, name='audio_samples')
        * False
        * None
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (3 total):
        * TensorSpec(shape=(None, None), dtype=tf.float32, name='inputs')
        * False
       

## Determine ideal number of epochs

In [ ]:
resnet_pretrained = pretrain_resnet()
dev_accs=np.zeros(40)
for idx1 in range(5):
  resnet_pretrained = pretrain_resnet()
  for idx in range(40):
    if idx%10==0:
      print(idx1, idx)
    history = resnet_pretrained.fit(specs_train_resnet, y_train, batch_size=64, epochs=1, verbose=0)
    _, train_acc = resnet_pretrained.evaluate(specs_train_resnet, y_train, verbose=0)
    _, dev_acc = resnet_pretrained.evaluate(specs_dev_resnet, y_dev, verbose=0)
    dev_accs[idx]= dev_accs[idx]+dev_acc

dev_accs = dev_accs/5

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(np.arange(40), dev_accs, color='orange', label='Accuracy on the Development Set')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuarcy')
plt.hlines(np.max(dev_accs), 0, np.argmax(dev_accs), colors='grey', linestyles='dashed')
plt.vlines(np.argmax(dev_accs), np.min(dev_accs), np.max(dev_accs), colors='grey', linestyles='dashed')
plt.legend(loc='lower right')
plt.show()

num_epochs = np.argmax(dev_accs)+1
print(num_epochs)

## Test Pretrained ResNet with optimal Parameters

In [ ]:
accs_dev_ResNet  = []
accs_test_ResNet = []
fscore_eval_ResNet   = []

for idx in range(100):
  print(idx)
  ResNet = pretrain_resnet()
  ResNet.fit(specs_train_resnet, y_train, batch_size=64, epochs=num_epochs, verbose=0)
  _, acc_dev_ger = ResNet.evaluate(specs_dev_ger_resnet, y_dev_ger, verbose=0)
  _, acc_dev_jap = ResNet.evaluate(specs_dev_jap_resnet, y_dev_jap, verbose=0)
  _, acc_test_ger = ResNet.evaluate(specs_test_ger_resnet, y_test_ger, verbose=0)
  _, acc_test_jap = ResNet.evaluate(specs_test_jap_resnet, y_test_jap, verbose=0)
  _, acc_eval = ResNet.evaluate(specs_eval_resnet, y_eval, verbose=0)
  acc_dev  = (acc_dev_ger+acc_dev_jap)/2
  acc_test = (acc_test_ger+acc_test_jap)/2
  accs_dev_ResNet.append(acc_dev)
  accs_test_ResNet.append(acc_test)
  fscore_eval_ResNet.append(2*(acc_eval)/(1+acc_eval))

save(accs_dev_ResNet,"/content/drive/My Drive/infant_cries/data_js_final/accs_dev_ResNet.json")
save(accs_test_ResNet,"/content/drive/My Drive/infant_cries/data_js_final/accs_test_ResNet.json")
save(fscore_eval_ResNet,"/content/drive/My Drive/infant_cries/data_js_final/fscore_eval_ResNet.json")

In [ ]:
accs_dev_ResNet      = load("/content/drive/My Drive/infant_cries/data_js_final/accs_dev_ResNet.json")
accs_test_ResNet     = load ("/content/drive/My Drive/infant_cries/data_js_final/accs_test_ResNet.json")
fscore_eval_ResNet   = load("/content/drive/My Drive/infant_cries/data_js_final/fscore_eval_ResNet.json")

In [ ]:
print(f"Mean:{np.mean(accs_dev_ResNet)}, Min:{np.min(accs_dev_ResNet)}, Max:{np.max(accs_dev_ResNet)}")
print(f"50th percentile. {np.percentile(accs_dev_ResNet, 50)}")
plt.hist(accs_dev_ResNet)
plt.axvline(np.mean(accs_dev_ResNet), color='red')
plt.show()

In [ ]:
print(f"Mean:{np.mean(accs_test_ResNet)}, Min:{np.min(accs_test_ResNet)}, Max:{np.max(accs_test_ResNet)}")
plt.hist(accs_test_ResNet)
plt.axvline(np.mean(accs_test_ResNet), color='red')
plt.show()

In [ ]:
print(f"Mean:{np.mean(fscore_eval_ResNet)}, Min:{np.min(fscore_eval_ResNet)}, Max:{np.max(fscore_eval_ResNet)}")
plt.hist(fscore_eval_ResNet)
plt.axvline(np.mean(fscore_eval_ResNet), color='red')
plt.show()